In [1]:
1+1

2

In [2]:
import os
%pwd

'e:\\MLOPS\\TextSummarizer\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\MLOPS\\TextSummarizer'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_fil: Path
    unzip_dir: Path

In [24]:
from src.textSummarizer.constants import*
from src.textSummarizer.utils.common import read_yaml,create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        
        
    
                

In [26]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger


### COMPONENTS

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"File downloaded successfully")
        else:
            logger.info("File already exists")

